# 🔬 KAN GridSearch - SAME AS RESNET-101

**Fair Comparison - Exact Same Parameters:**
1. ✅ **Optimizer**: Adam, AdamW, Adagrad
2. ✅ **Grid Size**: [3, 5] (KAN-specific)
3. ✅ **L1**: [0] (same as ResNet)
4. ✅ **L2**: [0, 1e-4, 1e-3] (same as ResNet)
5. ✅ **Early Stopping**: patience=10
6. ✅ **LR Scheduler**: CosineAnnealingLR
7. ✅ **Loss**: SoftFocalLoss (gamma=3.0)
8. ✅ **Data**: Hybrid loading
9. ✅ **CV**: 3-fold

**Total: 18 configs × 3 folds = 54 runs (~13 hours)**

## 📦 CELL 1: Setup & Imports

In [1]:
import os
from pathlib import Path
import random
import time
import gc
import json
import warnings
from datetime import datetime
from itertools import product
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print("="*80)
print(" KAN GridSearch - Fair Comparison with ResNet-101 ".center(80, "="))
print("="*80)
print(f"\nStarted: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("✅ Seed: 42")

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

# Paths
DATA_PKG = Path("data_package")
SPEC_DIR = Path("spec_hr_out")
RESULTS_DIR = Path("kan_gridsearch_results")
RESULTS_DIR.mkdir(exist_ok=True)

print(f"\n✅ Results: {RESULTS_DIR}")

=============== KAN GridSearch - Fair Comparison with ResNet-101 ===============

Started: 2026-01-12 15:12:33
✅ Seed: 42
✅ Device: cuda:0
   GPU: NVIDIA GeForce RTX 5060 Ti

✅ Results: kan_gridsearch_results


## 📊 CELL 2: Load Data

In [2]:
meta_use = pd.read_csv(DATA_PKG / "meta_use.csv")
lbl = np.load(DATA_PKG / "labels.npz", allow_pickle=True)
y_soft = lbl["y_soft"]
w_conf = lbl["w_conf"]
classes = [str(c) for c in lbl["classes"]]
y_hard = y_soft.argmax(axis=1)

print("✅ Data loaded")
print(f"   Samples: {len(y_hard)}")
print(f"   Classes: {classes}")

# 3-fold CV (SAME AS RESNET)
N_FOLDS = 3
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
folds = list(skf.split(meta_use, y_hard))
print(f"\n✅ Created {N_FOLDS}-fold CV")

✅ Data loaded
   Samples: 17089
   Classes: ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

✅ Created 3-fold CV


## 🤖 CELL 3: Dataset Class

In [13]:
class SpecDataset(Dataset):
    def __init__(self, df, root_dir, y_soft, w_conf, F_target=81, T_target=600):
        self.df = df.reset_index(drop=True)
        self.root = Path(root_dir)
        self.y_soft = y_soft
        self.w_conf = w_conf
        self.F_target = F_target
        self.T_target = T_target

    def __len__(self):
        return len(self.df)

    def _center_crop_pad(self, x):
        C, F, T = x.shape
        if F >= self.F_target:
            f0 = (F - self.F_target) // 2
            x = x[:, f0:f0+self.F_target, :]
        else:
            pad = self.F_target - F
            x = np.pad(x, ((0,0),(pad//2, pad-pad//2),(0,0)), mode="constant")
        if T >= self.T_target:
            t0 = (T - self.T_target) // 2
            x = x[:, :, t0:t0+self.T_target]
        else:
            pad = self.T_target - T
            x = np.pad(x, ((0,0),(0,0),(pad//2, pad-pad//2)), mode="constant")
        return x.copy()

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        eid = int(row.eeg_id)
        
        npz = np.load(self.root / f"{eid}_hr.npz")
        x = npz["x"]
        x = self._center_crop_pad(x)
        x = torch.from_numpy(x).float()
        x = F.interpolate(x.unsqueeze(0), size=(224, 224),
                          mode="bilinear", align_corners=False).squeeze(0)
        
        y = torch.from_numpy(self.y_soft[self.df.index[idx]]).float()
        w = torch.tensor(self.w_conf[self.df.index[idx]], dtype=torch.float32)
        
        return x, y, w

print("✅ Dataset ready")

✅ Dataset ready


## 🏗️ CELL 4: KAN Model

In [14]:
class KANLayer(nn.Module):
    def __init__(self, in_features, out_features, grid_size=5, spline_order=3):
        super().__init__()
        self.coefficients = nn.Parameter(torch.randn(out_features, in_features, grid_size) * 0.1)
        self.base_weight = nn.Parameter(torch.randn(out_features, in_features) * 0.1)
        self.register_buffer('grid', torch.linspace(-1, 1, grid_size))
        self.spline_order = spline_order
    
    def b_spline_basis(self, x):
        # Simplified B-spline using ReLU approximation
        x = x.unsqueeze(-1)  # (batch, in_features, 1)
        grid = self.grid.view(1, 1, -1)  # (1, 1, grid_size)
        distances = torch.abs(x - grid)
        basis = torch.relu(1 - distances) ** self.spline_order
        return basis / (basis.sum(dim=-1, keepdim=True) + 1e-8)
    
    def forward(self, x):
        basis = self.b_spline_basis(x)  # (batch, in_features, grid_size)
        spline_output = torch.einsum('bik,oik->bo', basis, self.coefficients)
        base_output = F.linear(x, self.base_weight)
        return spline_output + base_output


class KAN_4Ch(nn.Module):
    """KAN with CNN backbone for 4-channel EEG spectrograms"""
    
    def __init__(self, n_classes=6, n_channels=4, grid_size=5, spline_order=3):
        super().__init__()
        
        # CNN feature extractor
        self.features = nn.Sequential(
            nn.Conv2d(n_channels, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            
            nn.Conv2d(128, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # KAN classifier layers
        self.kan1 = KANLayer(256, 128, grid_size, spline_order)
        self.dropout1 = nn.Dropout(0.5)
        
        self.kan2 = KANLayer(128, 64, grid_size, spline_order)
        self.dropout2 = nn.Dropout(0.3)
        
        self.output = nn.Linear(64, n_classes)
        
        self.grid_size = grid_size
        self.spline_order = spline_order
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Extract features
        x = self.features(x)
        x = x.view(x.size(0), -1)
        
        # KAN layers
        x = self.kan1(x)
        x = self.dropout1(x)
        x = self.kan2(x)
        x = self.dropout2(x)
        
        # Output
        x = self.output(x)
        return x

print("✅ KAN model ready (SIMPLIFIED VERSION)")

✅ KAN model ready (SIMPLIFIED VERSION)


## 🎯 CELL 5: SoftFocalLoss

In [15]:
class SoftFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=3.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, logits, soft_targets, sample_weights=None):
        hard_targets = soft_targets.argmax(dim=1)
        probs = F.softmax(logits, dim=1)
        p_t = probs.gather(1, hard_targets.unsqueeze(1)).squeeze(1)
        ce_loss = -(soft_targets * F.log_softmax(logits, dim=1)).sum(dim=1)
        focal_weight = ((1 - p_t) ** self.gamma)
        loss = focal_weight * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[hard_targets]
            loss = alpha_t * loss
        
        if sample_weights is not None:
            loss = loss * sample_weights
        
        return loss.mean()

print("✅ SoftFocalLoss ready")

✅ SoftFocalLoss ready


## 📦 CELL 6: Hybrid Data Loader

In [16]:
def create_hybrid_loader(fold=0, target_ratio=0.4, weight_power=3.0, batch_size=16):
    tr_idx, va_idx = folds[fold]
    df_tr = meta_use.iloc[tr_idx]
    y_soft_tr, w_conf_tr = y_soft[tr_idx], w_conf[tr_idx]
    
    y_hard = y_soft_tr.argmax(axis=1)
    counts = np.bincount(y_hard, minlength=6)
    target = int(counts.max() * target_ratio)
    
    indices_add = []
    for i in range(6):
        mask = y_hard == i
        if mask.sum() < target:
            idx = np.where(mask)[0]
            n_add = target - mask.sum()
            indices_add.extend(np.random.choice(idx, n_add, replace=True))
    
    all_idx = np.concatenate([np.arange(len(y_hard)), indices_add])
    np.random.shuffle(all_idx)
    
    df_tr_over = df_tr.iloc[all_idx].reset_index(drop=True)
    y_soft_over, w_conf_over = y_soft_tr[all_idx], w_conf_tr[all_idx]
    
    y_hard_over = y_soft_over.argmax(axis=1)
    counts_over = np.bincount(y_hard_over, minlength=6)
    
    weights = (len(y_hard_over) / (counts_over + 1)) ** weight_power
    weights = torch.FloatTensor(weights / weights.sum() * 6)
    
    sample_weights = weights[y_hard_over].numpy()
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    ds_tr = SpecDataset(df_tr_over, SPEC_DIR, y_soft_over, w_conf_over)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, sampler=sampler, num_workers=0)
    
    ds_va = SpecDataset(meta_use.iloc[va_idx], SPEC_DIR, y_soft[va_idx], w_conf[va_idx])
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return dl_tr, dl_va, weights

print("✅ Hybrid loader ready")

✅ Hybrid loader ready


## 📈 CELL 7: Evaluation

In [17]:
@torch.no_grad()
def evaluate_full(model, loader):
    model.eval()
    preds, targets = [], []
    
    for x, y, w in loader:
        x = x.to(device)
        logits = model(x)
        preds.append(logits.argmax(1).cpu().numpy())
        targets.append(y.argmax(1).cpu().numpy())
    
    y_pred = np.concatenate(preds)
    y_true = np.concatenate(targets)
    
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
    }

print("✅ Evaluation ready")

✅ Evaluation ready


## 🏋️ CELL 8: Training Function

In [18]:
def train_one_config(fold, optimizer_name, grid_size, spline_order, l1_lambda, l2_lambda,
                     lr=3e-4, batch_size=16, epochs=30, patience=10):
    import sys
    
    # Data
    print(f"      [1/5] Data...", end=" ", flush=True)
    t0 = time.time()
    dl_tr, dl_va, class_weights = create_hybrid_loader(fold=fold, batch_size=batch_size)
    print(f"✓ ({time.time()-t0:.1f}s)", flush=True)
    
    # Model - FIXED: Use KAN_4Ch instead of EEGNet_KAN
    print(f"      [2/5] Model (grid={grid_size}, spline={spline_order})...", end=" ", flush=True)
    t0 = time.time()
    model = KAN_4Ch(
        n_classes=6, 
        n_channels=4,
        grid_size=grid_size, 
        spline_order=spline_order
    ).to(device)
    print(f"✓ ({time.time()-t0:.1f}s)", flush=True)
    
    # Optimizer
    print(f"      [3/5] Optimizer ({optimizer_name}, L2={l2_lambda:.0e})...", end=" ", flush=True)
    if optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda)
    elif optimizer_name == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2_lambda)
    elif optimizer_name == 'adagrad':
        optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=l2_lambda)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")
    print(f"✓", flush=True)
    
    # Loss & Scheduler
    print(f"      [4/5] Loss & Scheduler...", end=" ", flush=True)
    criterion = SoftFocalLoss(alpha=class_weights.to(device), gamma=3.0)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    print(f"✓", flush=True)
    
    # Training
    print(f"      [5/5] Training (patience={patience}, L1={l1_lambda:.0e})...", flush=True)
    best_f1, best_state, no_improve = 0.0, None, 0
    
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss, n = 0.0, 0
        
        for x, y, w in dl_tr:
            x, y, w = x.to(device), y.to(device), w.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y, w)
            
            # L1 Regularization (SAME AS RESNET)
            if l1_lambda > 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss = loss + l1_lambda * l1_norm
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item() * x.size(0)
            n += x.size(0)
        
        train_loss /= n
        val_results = evaluate_full(model, dl_va)
        scheduler.step()
        
        # Early stopping
        if val_results['f1'] > best_f1:
            best_f1 = val_results['f1']
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"        Early stop at epoch {epoch}", flush=True)
                break
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"        Epoch {epoch:2d}: F1={val_results['f1']:.4f}, Loss={train_loss:.4f}", flush=True)
        
        if epoch % 5 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    if best_state:
        model.load_state_dict(best_state)
    
    final_results = evaluate_full(model, dl_va)
    
    del model, optimizer, scheduler, dl_tr, dl_va
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_results

print("✅ Training function ready (KAN_4Ch)")
print("   Optimizers: Adam, AdamW, Adagrad")
print("   L1/L2 regularization supported")
print("   Early stopping: patience=10")

✅ Training function ready (KAN_4Ch)
   Optimizers: Adam, AdamW, Adagrad
   L1/L2 regularization supported
   Early stopping: patience=10


## 🔍 CELL 9: Grid Configuration (SAME AS RESNET)

In [19]:
print("\n" + "="*80)
print(" KAN GRIDSEARCH - SAME AS RESNET-101 ".center(80, "="))
print("="*80)

# EXACT SAME PARAMS AS RESNET-101
param_grid = {
    'optimizer': ['adam', 'adamw', 'adagrad'],  # 3 - SAME
    'grid_size': [3, 5],                        # 2 - KAN-specific
    'spline_order': [3],                        # 1 - fix
    'l1_lambda': [0],                           # 1 - SAME (no L1)
    'l2_lambda': [0, 1e-4, 1e-3],              # 3 - SAME
}

fixed_params = {
    'lr': 3e-4,
    'batch_size': 16,
    'epochs': 30,
    'patience': 10,
}

keys = list(param_grid.keys())
values = list(param_grid.values())
combinations = list(product(*values))

print("\n📋 HYPERPARAMETER GRID:")
print("-"*80)
print(f"  Optimizer:     {param_grid['optimizer']}")
print(f"  Grid size:     {param_grid['grid_size']} (KAN-specific)")
print(f"  Spline order:  {param_grid['spline_order']}")
print(f"  L1 lambda:     {param_grid['l1_lambda']} (SAME AS RESNET)")
print(f"  L2 lambda:     {param_grid['l2_lambda']} (SAME AS RESNET)")

print("\n📊 GRIDSEARCH STATISTICS:")
print("-"*80)
print(f"  Total combinations: {len(combinations)}")
print(f"  Folds per config:   {N_FOLDS}")
print(f"  Total trainings:    {len(combinations) * N_FOLDS}")
print(f"  Est. time per run:  ~15 min")
print(f"  Est. total time:    ~{len(combinations) * N_FOLDS * 15 / 60:.1f} hours")

print("\n📝 ALL COMBINATIONS:")
print("-"*80)
for i, combo in enumerate(combinations, 1):
    params = dict(zip(keys, combo))
    print(f"  {i:2d}. {params['optimizer']:7s} + grid={params['grid_size']} + "
          f"spline={params['spline_order']} + L1={params['l1_lambda']:.0e} + L2={params['l2_lambda']:.0e}")

print("\n🎯 RESNET-101 BEST CONFIG (to beat):")
print("-"*80)
print("  Optimizer: Adagrad")
print("  Activation: ReLU")
print("  L1: 0")
print("  L2: 1e-3")
print("  F1: 0.5585")
print("  Accuracy: 0.5921")

print("\n⏱️  TIMELINE:")
print("-"*80)
current_time = datetime.now()
finish_time = current_time + pd.Timedelta(hours=len(combinations) * N_FOLDS * 15 / 60)
print(f"  Start:  {current_time.strftime('%Y-%m-%d %H:%M')}")
print(f"  Finish: {finish_time.strftime('%Y-%m-%d %H:%M')} (approx)")

print("\n💾 AUTO-SAVE:")
print("-"*80)
print(f"  {RESULTS_DIR}/kan_gridsearch_progress.json")
print(f"  {RESULTS_DIR}/kan_gridsearch_final.json")


===================== KAN GRIDSEARCH - SAME AS RESNET-101 ======================

📋 HYPERPARAMETER GRID:
--------------------------------------------------------------------------------
  Optimizer:     ['adam', 'adamw', 'adagrad']
  Grid size:     [3, 5] (KAN-specific)
  Spline order:  [3]
  L1 lambda:     [0] (SAME AS RESNET)
  L2 lambda:     [0, 0.0001, 0.001] (SAME AS RESNET)

📊 GRIDSEARCH STATISTICS:
--------------------------------------------------------------------------------
  Total combinations: 18
  Folds per config:   3
  Total trainings:    54
  Est. time per run:  ~15 min
  Est. total time:    ~13.5 hours

📝 ALL COMBINATIONS:
--------------------------------------------------------------------------------
   1. adam    + grid=3 + spline=3 + L1=0e+00 + L2=0e+00
   2. adam    + grid=3 + spline=3 + L1=0e+00 + L2=1e-04
   3. adam    + grid=3 + spline=3 + L1=0e+00 + L2=1e-03
   4. adam    + grid=5 + spline=3 + L1=0e+00 + L2=0e+00
   5. adam    + grid=5 + spline=3 + L1=0e+00 

## 🚀 CELL 10: Run GridSearch

In [20]:
all_results = []
start_time = time.time()

print("\n" + "="*80)
print(" STARTING GRIDSEARCH ".center(80, "="))
print("="*80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

for combo_idx, combo in enumerate(combinations, 1):
    params = dict(zip(keys, combo))
    
    print("\n" + "="*80)
    print(f" CONFIG {combo_idx}/{len(combinations)} ".center(80, "="))
    print("="*80)
    print(f"  Optimizer: {params['optimizer']}")
    print(f"  Grid size: {params['grid_size']}")
    print(f"  Spline: {params['spline_order']}")
    print(f"  L1: {params['l1_lambda']:.0e}")
    print(f"  L2: {params['l2_lambda']:.0e}")
    print("-"*80)
    
    fold_results = []
    
    for fold in range(N_FOLDS):
        print(f"\n    Fold {fold+1}/{N_FOLDS}...", flush=True)
        fold_start = time.time()
        
        try:
            result = train_one_config(
                fold=fold,
                optimizer_name=params['optimizer'],
                grid_size=params['grid_size'],
                spline_order=params['spline_order'],
                l1_lambda=params['l1_lambda'],
                l2_lambda=params['l2_lambda'],
                **fixed_params
            )
            fold_results.append(result)
            print(f"\n    ✓ Fold {fold+1}: F1={result['f1']:.4f} ({(time.time()-fold_start)/60:.1f} min)", flush=True)
        except Exception as e:
            print(f"\n    ❌ Error: {e}", flush=True)
            fold_results.append({'f1': 0.0, 'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0})
    
    mean_metrics = {
        'f1': np.mean([r['f1'] for r in fold_results]),
        'accuracy': np.mean([r['accuracy'] for r in fold_results]),
        'precision': np.mean([r['precision'] for r in fold_results]),
        'recall': np.mean([r['recall'] for r in fold_results]),
        'f1_std': np.std([r['f1'] for r in fold_results]),
    }
    
    result_entry = {
        'config_id': combo_idx,
        'params': params,
        'mean_metrics': mean_metrics,
        'fold_results': fold_results,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    }
    all_results.append(result_entry)
    
    print(f"\n  Mean F1: {mean_metrics['f1']:.4f} ± {mean_metrics['f1_std']:.4f}")
    print(f"  Mean Acc: {mean_metrics['accuracy']:.4f}")
    
    # Auto-save
    with open(RESULTS_DIR / 'kan_gridsearch_progress.json', 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    print(f"  💾 Saved", flush=True)

# Final save
with open(RESULTS_DIR / 'kan_gridsearch_final.json', 'w') as f:
    json.dump({
        'all_results': all_results,
        'param_grid': param_grid,
        'fixed_params': fixed_params,
        'total_time_hours': (time.time() - start_time) / 3600,
    }, f, indent=2, default=str)

print("\n" + "="*80)
print(" GRIDSEARCH COMPLETE ".center(80, "="))
print("="*80)
print(f"Total time: {(time.time()-start_time)/3600:.2f} hours")


============================= STARTING GRIDSEARCH ==============================
Started: 2026-01-12 15:30:32


================================= CONFIG 1/18 ==================================
  Optimizer: adam
  Grid size: 3
  Spline: 3
  L1: 0e+00
  L2: 0e+00
--------------------------------------------------------------------------------

    Fold 1/3...
      [1/5] Data... ✓ (0.0s)
      [2/5] Model (grid=3, spline=3)... ✓ (0.5s)
      [3/5] Optimizer (adam, L2=0e+00)... ✓
      [4/5] Loss & Scheduler... ✓
      [5/5] Training (patience=10, L1=0e+00)...
        Epoch  1: F1=0.2389, Loss=2.3359
        Epoch  5: F1=0.2874, Loss=0.8596
        Epoch 10: F1=0.3729, Loss=0.6621
        Epoch 15: F1=0.3706, Loss=0.5467
        Epoch 20: F1=0.3998, Loss=0.4420
        Epoch 25: F1=0.4011, Loss=0.3581
        Epoch 30: F1=0.3987, Loss=0.3371

    ✓ Fold 1: F1=0.4078 (34.7 min)

    Fold 2/3...
      [1/5] Data... ✓ (0.0s)
      [2/5] Model (grid=3, spline=3)... ✓ (0.0s)
      [3/5] Optim

## 📊 CELL 11: Analyze Results

In [22]:
sorted_results = sorted(all_results, key=lambda x: x['mean_metrics']['f1'], reverse=True)

print("\n" + "="*80)
print(" KAN GRIDSEARCH RESULTS ".center(80, "="))
print("="*80)

print("\n🏆 TOP 10 CONFIGURATIONS:")
print("="*80)
print(f"{'Rank':<6} {'Optimizer':>10} {'Grid':>6} {'Spline':>8} {'L2':>8} {'F1':>10} {'Acc':>8}")
print("-"*80)

for i, result in enumerate(sorted_results[:10], 1):
    p = result['params']
    m = result['mean_metrics']
    print(f"{i:<6} {p['optimizer']:>10} {p['grid_size']:>6} {p['spline_order']:>8} "
          f"{p['l2_lambda']:>8.0e} {m['f1']:>10.4f} {m['accuracy']:>8.4f}")

best = sorted_results[0]
print("\n" + "="*80)
print(" BEST KAN CONFIGURATION ".center(80, "="))
print("="*80)
print(f"  Optimizer:    {best['params']['optimizer']}")
print(f"  Grid size:    {best['params']['grid_size']}")
print(f"  Spline order: {best['params']['spline_order']}")
print(f"  L1:           {best['params']['l1_lambda']:.0e}")
print(f"  L2:           {best['params']['l2_lambda']:.0e}")
print(f"\n  F1:       {best['mean_metrics']['f1']:.4f} ± {best['mean_metrics']['f1_std']:.4f}")
print(f"  Accuracy: {best['mean_metrics']['accuracy']:.4f}")

print("\n" + "="*80)
print(" FINAL COMPARISON ".center(80, "="))
print("="*80)
print(f"  KAN (best):        F1 = {best['mean_metrics']['f1']:.4f}, Acc = {best['mean_metrics']['accuracy']:.4f}")
print(f"  ResNet-101 (best): F1 = 0.5585,  Acc = 0.5921")

if best['mean_metrics']['f1'] > 0.5585:
    print("\n  🎉 KAN WINS!")
elif best['mean_metrics']['f1'] > 0.4073:
    print(f"\n  ✅ KAN improved by {((best['mean_metrics']['f1'] - 0.4073) / 0.4073 * 100):.1f}%")
else:
    print("\n  ⚠️  KAN did not improve")

print("\n💾 Results saved to:")
print(f"   {RESULTS_DIR}/kan_gridsearch_final.json")


============================ KAN GRIDSEARCH RESULTS ============================

🏆 TOP 10 CONFIGURATIONS:
Rank    Optimizer   Grid   Spline       L2         F1      Acc
--------------------------------------------------------------------------------
1           adamw      5        3    1e-03     0.4104   0.3973
2           adamw      5        3    0e+00     0.4077   0.3909
3            adam      3        3    0e+00     0.4073   0.3950
4            adam      5        3    0e+00     0.4067   0.3900
5           adamw      3        3    1e-04     0.4058   0.3862
6            adam      3        3    1e-04     0.4057   0.3972
7           adamw      5        3    1e-04     0.4052   0.3871
8           adamw      3        3    1e-03     0.4048   0.3861
9           adamw      3        3    0e+00     0.4046   0.3888
10           adam      5        3    1e-04     0.4046   0.3890

============================ BEST KAN CONFIGURATION ============================
  Optimizer:    adamw
  Grid size:  

In [2]:
import json
from pathlib import Path
import pandas as pd

# Load file KAN results
file_path = r"C:\Users\numpppy\Downloads\hms-harmful-brain-activity-classification\kan_gridsearch_results\kan_gridsearch_final.json"

print("🔍 Loading KAN GridSearch results...")
print(f"📁 {file_path}\n")

try:
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Extract results (cek key-nya dulu)
    all_results = data.get('all_results', data.get('sorted_results', data.get('results', [])))
    
    if not all_results:
        print("❌ No results found in file!")
        print(f"Available keys: {list(data.keys())}")
    else:
        print(f"✅ Loaded {len(all_results)} configurations!\n")
        
        # Sort by F1
        sorted_results = sorted(all_results, key=lambda x: x['mean_metrics']['f1'], reverse=True)
        
        print("="*80)
        print(" KAN GRIDSEARCH RESULTS ".center(80, "="))
        print("="*80)
        
        print("\n🏆 TOP 10 CONFIGURATIONS:")
        print("="*80)
        print(f"{'Rank':<6} {'Optimizer':>10} {'Grid':>6} {'Spline':>8} {'L2':>8} "
              f"{'F1':>10} {'Acc':>8} {'Prec':>8} {'Rec':>8}")
        print("-"*80)
        
        for i, result in enumerate(sorted_results[:10], 1):
            p = result['params']
            m = result['mean_metrics']
            print(f"{i:<6} {p['optimizer']:>10} {p['grid_size']:>6} {p['spline_order']:>8} "
                  f"{p['l2_lambda']:>8.0e} {m['f1']:>10.4f} {m['accuracy']:>8.4f} "
                  f"{m.get('precision', 0):>8.4f} {m.get('recall', 0):>8.4f}")
        
        best = sorted_results[0]
        print("\n" + "="*80)
        print(" BEST KAN CONFIGURATION ".center(80, "="))
        print("="*80)
        print(f"  Config ID:    {best.get('config_id', 'N/A')}")
        print(f"  Optimizer:    {best['params']['optimizer']}")
        print(f"  Grid size:    {best['params']['grid_size']}")
        print(f"  Spline order: {best['params']['spline_order']}")
        print(f"  L1:           {best['params']['l1_lambda']:.0e}")
        print(f"  L2:           {best['params']['l2_lambda']:.0e}")
        
        m = best['mean_metrics']
        print(f"\n  📊 Performance Metrics:")
        print(f"     F1:        {m['f1']:.4f} ± {m.get('f1_std', 0):.4f}")
        print(f"     Accuracy:  {m['accuracy']:.4f} ± {m.get('accuracy_std', 0):.4f}")
        print(f"     Precision: {m.get('precision', 0):.4f} ± {m.get('precision_std', 0):.4f}")
        print(f"     Recall:    {m.get('recall', 0):.4f} ± {m.get('recall_std', 0):.4f}")
        if 'loss' in m:
            print(f"     Loss:      {m.get('loss', 0):.4f} ± {m.get('loss_std', 0):.4f}")
        
        print("\n" + "="*80)
        print(" FINAL COMPARISON ".center(80, "="))
        print("="*80)
        print(f"{'Model':<25} {'F1':>10} {'Accuracy':>10} {'Precision':>10} {'Recall':>10}")
        print("-"*80)
        print(f"{'KAN (best)':<25} {m['f1']:>10.4f} {m['accuracy']:>10.4f} "
              f"{m.get('precision', 0):>10.4f} {m.get('recall', 0):>10.4f}")
        print(f"{'ResNet-101 (best)':<25} {0.5585:>10.4f} {0.5921:>10.4f} "
              f"{'N/A':>10} {'N/A':>10}")
        
        # Improvement analysis
        kan_baseline = 0.4073  # Sesuaikan dengan baseline KAN kamu
        resnet_best = 0.5585
        
        print()
        if m['f1'] > resnet_best:
            improvement = ((m['f1'] - resnet_best) / resnet_best) * 100
            print(f"  🎉 KAN WINS! Outperformed ResNet-101 by {improvement:.1f}%!")
        elif m['f1'] > kan_baseline:
            improvement = ((m['f1'] - kan_baseline) / kan_baseline) * 100
            gap = ((resnet_best - m['f1']) / resnet_best) * 100
            print(f"  ✅ KAN improved by {improvement:.1f}% from baseline")
            print(f"  📉 Still {gap:.1f}% behind ResNet-101")
        else:
            decline = ((kan_baseline - m['f1']) / kan_baseline) * 100
            print(f"  ⚠️  KAN declined by {decline:.1f}% from baseline")
        
        # Hyperparameter analysis
        print("\n" + "="*80)
        print(" 📊 HYPERPARAMETER IMPACT ".center(80, "="))
        print("="*80)
        
        records = []
        for r in all_results:
            records.append({**r['params'], **r['mean_metrics']})
        df = pd.DataFrame(records)
        
        # Optimizer
        print("\n🔧 OPTIMIZER PERFORMANCE:")
        print("-"*80)
        opt_stats = df.groupby('optimizer')['f1'].agg(['mean', 'std', 'max', 'count'])
        print(opt_stats.sort_values('mean', ascending=False))
        
        # Grid size
        print("\n📐 GRID SIZE IMPACT:")
        print("-"*80)
        grid_stats = df.groupby('grid_size')['f1'].agg(['mean', 'std', 'max', 'count'])
        print(grid_stats.sort_values('mean', ascending=False))
        
        # Spline order
        print("\n📈 SPLINE ORDER IMPACT:")
        print("-"*80)
        spline_stats = df.groupby('spline_order')['f1'].agg(['mean', 'std', 'max', 'count'])
        print(spline_stats.sort_values('mean', ascending=False))
        
        # L2 regularization
        print("\n🎛️ L2 REGULARIZATION IMPACT:")
        print("-"*80)
        l2_stats = df.groupby('l2_lambda')['f1'].agg(['mean', 'std', 'max', 'count'])
        print(l2_stats.sort_values('mean', ascending=False))
        
        # Overall stats
        print("\n📈 OVERALL STATISTICS:")
        print("="*80)
        print(f"Total configs: {len(df)}")
        print(f"\nF1 Score:")
        print(f"  Best:   {df['f1'].max():.4f}")
        print(f"  Worst:  {df['f1'].min():.4f}")
        print(f"  Mean:   {df['f1'].mean():.4f}")
        print(f"  Median: {df['f1'].median():.4f}")
        print(f"  Std:    {df['f1'].std():.4f}")
        
        # Top by different metrics
        print("\n🏆 TOP 3 BY DIFFERENT METRICS:")
        print("-"*80)
        
        print("\nBy Precision:")
        top_prec = df.nlargest(3, 'precision')[['optimizer', 'grid_size', 'precision', 'f1']]
        print(top_prec.to_string(index=False))
        
        print("\nBy Recall:")
        top_rec = df.nlargest(3, 'recall')[['optimizer', 'grid_size', 'recall', 'f1']]
        print(top_rec.to_string(index=False))
        
        print("\n" + "="*80)
        print("✅ Analysis complete!")
        print("="*80)

except FileNotFoundError:
    print(f"❌ File not found: {file_path}")
    print("\n🔍 Let me search for it...")
    
    # Auto-search di Downloads
    downloads = Path.home() / "Downloads"
    kan_files = list(downloads.glob("**/kan*.json"))
    
    if kan_files:
        print(f"\nFound {len(kan_files)} KAN-related files:")
        for i, f in enumerate(kan_files, 1):
            print(f"  {i}. {f}")
        print("\nUpdate the file_path variable to the correct location!")
    else:
        print("\n❌ No KAN files found in Downloads")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

🔍 Loading KAN GridSearch results...
📁 C:\Users\numpppy\Downloads\hms-harmful-brain-activity-classification\kan_gridsearch_results\kan_gridsearch_final.json

✅ Loaded 18 configurations!

============================ KAN GRIDSEARCH RESULTS ============================

🏆 TOP 10 CONFIGURATIONS:
Rank    Optimizer   Grid   Spline       L2         F1      Acc     Prec      Rec
--------------------------------------------------------------------------------
1           adamw      5        3    1e-03     0.4104   0.3973   0.3327   0.5601
2           adamw      5        3    0e+00     0.4077   0.3909   0.3347   0.5556
3            adam      3        3    0e+00     0.4073   0.3950   0.3310   0.5508
4            adam      5        3    0e+00     0.4067   0.3900   0.3326   0.5571
5           adamw      3        3    1e-04     0.4058   0.3862   0.3372   0.5497
6            adam      3        3    1e-04     0.4057   0.3972   0.3268   0.5470
7           adamw      5        3    1e-04     0.4052   0.3

In [4]:
print("\n" + "="*80)
print(" KAN RESULTS ANALYSIS ".center(80, "="))
print("="*80)

# Sort by F1
sorted_results = sorted(all_results, key=lambda x: x['mean_metrics']['f1'], reverse=True)

print("\n🏆 TOP 10 CONFIGURATIONS:")
print("="*80)
print(f"{'Rank':<6} {'Optimizer':>10} {'Grid':>6} {'Spline':>8} {'L1':>8} {'L2':>8} {'F1':>10} {'Acc':>8}")
print("-"*80)

for i, result in enumerate(sorted_results[:10], 1):
    p = result['params']
    m = result['mean_metrics']
    print(f"{i:<6} {p['optimizer']:>10} {p['grid_size']:>6} {p['spline_order']:>8} "
          f"{p['l1_lambda']:>8.0e} {p['l2_lambda']:>8.0e} "
          f"{m['f1']:>10.4f} {m['accuracy']:>8.4f}")

# Best config
best_result = sorted_results[0]
best_params = best_result['params']
best_metrics = best_result['mean_metrics']

print("\n" + "="*80)
print(" BEST CONFIGURATION ".center(80, "="))
print("="*80)

print("\n📋 Best Hyperparameters:")
print(f"  Optimizer:    {best_params['optimizer']}")
print(f"  Grid size:    {best_params['grid_size']}")
print(f"  Spline order: {best_params['spline_order']}")
print(f"  L1 lambda:    {best_params['l1_lambda']:.0e}")
print(f"  L2 lambda:    {best_params['l2_lambda']:.0e}")

print("\n📊 Best Performance:")
print(f"  F1 Score:   {best_metrics['f1']:.4f} ± {best_metrics['f1_std']:.4f}")
print(f"  Accuracy:   {best_metrics['accuracy']:.4f}")
print(f"  Precision:  {best_metrics['precision']:.4f}")
print(f"  Recall:     {best_metrics['recall']:.4f}")

# Comparison
print("\n📈 COMPARISON WITH BASELINES:")
print("-"*80)
print(f"  EEGNet:    F1 = 0.3281")
print(f"  KAN (Best):       F1 = {best_metrics['f1']:.4f}")
print(f"  ResNet-101: F1 = 0.5585")

if best_metrics['f1'] > 0.5585:
    improvement = ((best_metrics['f1'] - 0.5585) / 0.5585) * 100
    print(f"\n  🎉 KAN BEATS ResNet-101 by {improvement:.1f}%!")
elif best_metrics['f1'] > 0.3281:
    improvement_eegnet = ((best_metrics['f1'] - 0.3281) / 0.3281) * 100
    gap_resnet = ((0.5585 - best_metrics['f1']) / 0.5585) * 100
    print(f"\n  ✅ KAN beats EEGNet by {improvement_eegnet:.1f}%")
    print(f"  📉 KAN is {gap_resnet:.1f}% below ResNet-101")
else:
    gap = ((0.3281 - best_metrics['f1']) / 0.3281) * 100
    print(f"\n  ⚠️  KAN is {gap:.1f}% below EEGNet baseline")

print("\n" + "="*80)
print(" ALL RESULTS SAVED ".center(80, "="))
print("="*80)
print(f"\n📁 Results directory: {RESULTS_DIR}/")
print(f"   - kan_gridsearch_final.json (complete results)")
print(f"   - kan_gridsearch_summary.txt (readable summary)")
print(f"   - kan_gridsearch_progress.json (backup)")

print("\n" + "="*80)


============================= KAN RESULTS ANALYSIS =============================

🏆 TOP 10 CONFIGURATIONS:
Rank    Optimizer   Grid   Spline       L1       L2         F1      Acc
--------------------------------------------------------------------------------
1           adamw      5        3    0e+00    1e-03     0.4104   0.3973
2           adamw      5        3    0e+00    0e+00     0.4077   0.3909
3            adam      3        3    0e+00    0e+00     0.4073   0.3950
4            adam      5        3    0e+00    0e+00     0.4067   0.3900
5           adamw      3        3    0e+00    1e-04     0.4058   0.3862
6            adam      3        3    0e+00    1e-04     0.4057   0.3972
7           adamw      5        3    0e+00    1e-04     0.4052   0.3871
8           adamw      3        3    0e+00    1e-03     0.4048   0.3861
9           adamw      3        3    0e+00    0e+00     0.4046   0.3888
10           adam      5        3    0e+00    1e-04     0.4046   0.3890

==================

NameError: name 'RESULTS_DIR' is not defined